In [ ]:
#@title Install Python 3.12 with Miniconda
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py312_24.11.1-0-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py312" --user
!python --version

In [ ]:
#@title Download Repos
!sudo apt update
!sudo apt install git-lfs
!git lfs install
!git clone https://github.com/sgsdxzy/YuE-exllamav2.git
%cd /content/YuE-exllamav2
!git clone https://huggingface.co/m-a-p/xcodec_mini_infer

In [ ]:
#@title Requirements
%cd /content/YuE-exllamav2
!pip install torch==2.5.1 omegaconf torchaudio einops numpy transformers sentencepiece tqdm tensorboard descript-audiotools>=0.7.2 descript-audio-codec scipy>=1.10.1 accelerate>=0.26.0
!wget https://github.com/turboderp-org/exllamav2/releases/download/v0.2.7/exllamav2-0.2.7+cu121.torch2.5.0-cp312-cp312-linux_x86_64.whl
!pip install exllamav2-0.2.7+cu121.torch2.5.0-cp312-cp312-linux_x86_64.whl
!rm -r exllamav2-0.2.7+cu121.torch2.5.0-cp312-cp312-linux_x86_64.whl
import os
import torch

# Check for CUDA availability
if torch.cuda.is_available():
    major, minor = torch.cuda.get_device_capability()
    if major >= 8:
        print(f"Ampere+ GPU detected (sm_{major}{minor}) - installing flash-attn")
        !pip install flash-attn
    else:
        print(f"Pre-Ampere GPU detected (sm_{major}{minor}) - skipping installation of flash-attn")
else:
    print("No GPU available - skipping installation of flash-attn")

In [ ]:
#@title Model downloader
%cd /content
model = "YuE-s1-7B-anneal-en-cot" # @param ["YuE-s1-7B-anneal-en-cot","YuE-s1-7B-anneal-jp-kr-cot","YuE-s1-7B-anneal-zh-cot", "YuE-s1-7B-anneal-en-icl","YuE-s1-7B-anneal-jp-kr-icl","YuE-s1-7B-anneal-zh-icl"]
!git clone https://huggingface.co/m-a-p/{model}
model_stage2 = "https://huggingface.co/m-a-p/YuE-s2-1B-general"
!git clone {model_stage2}

In [ ]:
#@title Run!
%cd /content/YuE-exllamav2
genre_path = "/content/YuE-exllamav2/prompt_egs/genre.txt" #@param {type:"string"}
lyrics_path = "/content/YuE-exllamav2/prompt_egs/lyrics.txt" #@param {type:"string"}
output_path = "/content/YuE-exllamav2/output" #@param {type:"string"}
s1_folder = "/content/YuE-s1-7B-anneal-en-cot" # @param {"type":"string"}
s2_folder = "/content/YuE-s2-1B-general" # @param {"type":"string"}
!python src/yue/infer.py --stage1_use_exl2 --stage2_use_exl2 --stage2_cache_size 32768 \
    --cuda_idx 0 \
    --stage1_model {s1_folder} \
    --stage2_model {s2_folder} \
    --genre_txt '{genre_path}' \
    --lyrics_txt '{lyrics_path}' \
    --run_n_segments 2 \
    --stage2_batch_size 4 \
    --output_dir '{output_path}' \
    --max_new_tokens 3000

# **Want more advanced stuff?**

In [ ]:
#@title Run! (Single Audio ICL Mode)
%cd /content/YuE-exllamav2
genre_path = "/content/YuE-exllamav2/prompt_egs/genre.txt" #@param {type:"string"}
lyrics_path = "/content/YuE-exllamav2/prompt_egs/lyrics.txt" #@param {type:"string"}
output_path = "/content/YuE-exllamav2/output" #@param {type:"string"}
s1_folder = "/content/YuE-s1-7B-anneal-en-cot" # @param {"type":"string"}
s2_folder = "/content/YuE-s2-1B-general" # @param {"type":"string"}
start_sec = 0 # @param {"type":"integer"}
end_sec = 30 # @param {"type":"integer"}
audio_path = "/content/YuE-exllamav2/prompt_egs/audio.wav" # @param {"type":"string"}
!python src/yue/infer.py --stage1_use_exl2 --stage2_use_exl2 --stage2_cache_size 32768 \
    --cuda_idx 0 \
    --stage1_model {s1_folder} \
    --stage2_model {s2_folder} \
    --genre_txt '{genre_path}'  \
    --lyrics_txt '{lyrics_path}' \
    --run_n_segments 2 \
    --stage2_batch_size 4 \
    --output_dir '{output_path}' \
    --use_audio_prompt \
    --audio_prompt_path '{audio_path}' \
    --prompt_start_time {start_sec} \
    --prompt_end_time {end_sec}

In [ ]:
#@title Run! (Dual Audio ICL Mode)
%cd /content/YuE-exllamav2
genre_path = "/content/YuE-exllamav2/prompt_egs/genre.txt" #@param {type:"string"}
lyrics_path = "/content/YuE-exllamav2/prompt_egs/lyrics.txt" #@param {type:"string"}
output_path = "/content/YuE-exllamav2/output" #@param {type:"string"}
s1_folder = "/content/YuE-s1-7B-anneal-en-icl" # @param {"type":"string"}
s2_folder = "/content/YuE-s2-1B-general" # @param {"type":"string"}
start_sec = 0 # @param {"type":"integer"}
end_sec = 27 # @param {"type":"integer"}
vocal_path = "" # @param {"type":"string"}
inst_path = "" # @param {"type":"string"}
!python src/yue/infer.py --stage1_use_exl2 --stage2_use_exl2 --stage2_cache_size 32768 \
    --cuda_idx 0 \
    --stage1_model {s1_folder} \
    --stage2_model {s2_folder} \
    --genre_txt '{genre_path}'  \
    --lyrics_txt '{lyrics_path}' \
    --run_n_segments 2 \
    --stage2_batch_size 2 \
    --output_dir '{output_path}' \
    --use_dual_tracks_prompt \
    --vocal_track_prompt_path '{vocal_path}' \
    --instrumental_track_prompt_path '{inst_path}' \
    --prompt_start_time {start_sec} \
    --prompt_end_time {end_sec}